In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
def Prediction(songname,model):
    y,sr = librosa.load(songname, mono=True, duration=30)
    rmse = librosa.feature.rms(y=y)[0]
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = [np.mean(chroma_stft),np.mean(rmse),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr)]    
    for e in mfcc:
          to_append.append(np.mean(e))
  
    data = pd.DataFrame([to_append])
    data.astype('float64')
    #Encoding the Labels
    #genre_list = data.iloc[:, -1]
    #encoder = LabelEncoder()
    #y = encoder.fit_transform(genre_list)
    #Scaling the Feature columns
    scaler = StandardScaler()
    X = scaler.fit_transform(np.array(data, dtype = float))
    #load and create model
    from sklearn.externals import joblib
    model_load = joblib.load(model)

    pred=model_load.predict(X)
    #argmax() method 
    pred = [np.argmax(line) for line in pred]
    dic ={0:'blues', 1:'classical', 2:'country', 3:'disco', 4:'hiphop', 5:'jazz',
       6:'metal', 7:'pop', 8:'reggae', 9:'rock'}
    pred = [dic.get(n, n) for n in pred]
    
    return pred